# **Buy Till You Die Models: Customer Lifetime Value**

<p><img style="float: left;margin:20px 10px 5px 1px; max-width:380px" src="https://i.ytimg.com/vi/l_xwYM2KNX0/maxresdefault.jpg"></p>
<p> The Buy Till You Die (BTYD) class of statistical models is designed to predict the behavioral characteristics of customers. The main purpose is to model and forecast customer lifetime value. </p>
<p> Customer lifetime value (CLTV or CLV) can be defined as the present value of a customer for the company based on projected future cash flows from the customer relationship. CLTV represents the total amount of money spent on the business or products over lifetime of a customer.</p>
<p> In this notebook, I'll try to apply a BTYD model on an online retail dataset to make a 1-year CLTV prediction.</p> 

## 1. Loading the Dataset & Checking Variables


### ****Content of Dataset****

This Online Retail II data set contains all the transactions occurring for a UK-based and registered, non-store online retail between 01/12/2009 and 09/12/2011.The company mainly sells unique all-occasion gift-ware. Many customers of the company are wholesalers.

### ****Definiton of Variables****
1. **Invoice:** Invoice number, unique identifier variable for each transaction. Refund invoice numbers starts with "C"
2. **StockCode:** Unique product code
3. **Description:** Product name
4. **Quantity:** The number of product in the invoice
5. **InvoiceDate:** Date and time of the purchase
6. **Price:** Unit price of a product (in terms of Sterlin)
7. **CustomerID:** Unique customer identifier
8. **Country:** Residential country of customers

In [ ]:
pip install openpyxl

In [ ]:
pip install xlrd

In [ ]:
# Loading the required libraries
import datetime as dt
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
# Reading the online retail dataset
df_ = pd.read_excel("../input/online-retail-ii-data-set-from-ml-repository/online_retail_II.xlsx",sheet_name="Year 2010-2011")

# Copying the online retail dataset 
df = df_.copy()
df.head()

In [ ]:
# Checking numerical variables
df.describe().T

<p> When we look at the descriptive statistics of numerical variables, we can see that there are negative values in the Quantity and Price columns. These transactions are canceled orders. In the next step (data preprocessing), we'll eliminate these observations.

In [ ]:
# Checking null variables 
df.isna().sum()

There are null observations in the dataset, especially in the Customer ID column. Since this analysis will be consumer-based, we need to remove these observations from the dataset. Therefore, we will  eliminate these observations in the next step (data preprocessing).

## 2. Data Preprocessing

In [ ]:
# Data preparation step 1: Removing null oberservations
df.dropna(inplace=True)

# Data preparation step 1: Removing canceled orders 
df = df[~df["Invoice"].str.contains("C", na=False)]
df = df[df["Quantity"] > 0]

df.describe([0.01,0.25,0.50,0.75,0.99]).T

<p>Check out descriptive statistics of numerical variables again! Look at 99% quantile of Quantity and Price columns and compare with the maximum values. We can say that there are some outliers.</p>
<p>Let's find out these outliers and replace them with the highest limit.</p>

In [ ]:
# Defining functions for outliers
def outlier_thresholds(dataframe, variable):
    quartile1 = dataframe[variable].quantile(0.01)
    quartile3 = dataframe[variable].quantile(0.99)
    interquantile_range = quartile3 - quartile1
    up_limit = quartile3 + 1.5 * interquantile_range
    low_limit = quartile1 - 1.5 * interquantile_range
    return low_limit, up_limit

# Defining functions to replace outliers
def replace_with_thresholds(dataframe, variable):
    low_limit, up_limit = outlier_thresholds(dataframe, variable)
    # dataframe.loc[(dataframe[variable] < low_limit), variable] = low_limit
    dataframe.loc[(dataframe[variable] > up_limit), variable] = up_limit

In [ ]:
# Data preparation step 2: Replacing outliers in the Quantity and Price columns with the upper limit
replace_with_thresholds(df, "Quantity")
replace_with_thresholds(df, "Price")

df.describe([0.01,0.25,0.50,0.75,0.99]).T

In [ ]:
# Data preparation step 3: Calculating total price per transaction 
df["TotalPrice"] = df["Quantity"] * df["Price"]
today_date = dt.datetime(2011, 12, 11)

In [ ]:
# Defining today date as max(InvoiceDate) + 2 days
today_date = dt.datetime(2011, 12, 11)
print(f" Maximum invoice date: {df.InvoiceDate.max()} \n Today date: {today_date}")

## 3. Deriving the RFM Metrics

<p><img style="float: right;margin:-10px 20px 20px 5px; max-width:380px" src="https://www.retailreco.com/blog/wp-content/uploads/2018/11/RFM-Analytics.jpg"></p>

The predictive CLTV models are built around 4 key metrics. These are:

**<p>Recency:**  The age of the customer at the time of their last purchase.
**<p>Monetary:** The average total sales of the customer.
**<p>Frequency:** Number of purchases/transactions.
**<p>Age (T):** The age of the customer since the date of a customer's first purchase to the current date.

In [ ]:
# Calculating recency, monetary, frequency and tenure metrics
rfm = df.groupby("Customer ID").agg({"InvoiceDate": [lambda date: (date.max() - date.min()).days,
                                                     lambda date: (today_date - date.min()).days],
                                     "Invoice": lambda num: num.nunique(),
                                      "TotalPrice": lambda price: price.sum()}) #total price per customer

rfm.columns = rfm.columns.droplevel(0)
rfm.columns = ['Recency', "T", 'Frequency', "Monetary"]

# Calculating average monetary values per order:
rfm["Monetary"] = rfm["Monetary"] / rfm["Frequency"]

rfm.head()

In [ ]:
# Removing one-time purchases from dataset
rfm = rfm[(rfm['Frequency'] > 1)]

# Copying dataset
cltv = rfm.copy()
rfm.head()

# 4. Train BG-NBD Model (Expected number of transaction)

In this step, we will train the BG/NBD model to estimate the expected sales in a given period of time.
Also, the trained BG/NBD model will be used in predicting customer lifetime value in the following steps. 

In [ ]:
pip install lifetimes

In [ ]:
# Loading the required libraries
from lifetimes import BetaGeoFitter
from lifetimes import GammaGammaFitter
from lifetimes.plotting import plot_probability_alive_matrix

In [ ]:
# Checking BG/NBD model assumption and requirements
print(cltv[['Monetary', 'Recency']].corr())  # Correlation between monetary ve recency variables
cltv["Frequency"] = cltv["Frequency"].astype(int) # Type of frequency variable should be integer for BG-NBD model

In [ ]:
# Creating BG-NBD Model
bgf = BetaGeoFitter(penalizer_coef=0.001) # model object
bgf.fit(cltv['Frequency'], cltv['Recency'], cltv['T']) # model fitting

# Prediction of expected number of transaction for each customer for one year (365 days)
cltv['expctd_num_of_purch'] = bgf.predict(365, cltv['Frequency'], cltv['Recency'], cltv['T']) 
cltv.sort_values("expctd_num_of_purch",ascending=False).head()

Look at the new columns that we estimate average spending for each customer in a year. By using this information, you can make many business decisions!

This model can also estimate the retention rate and the probability of the customer is alive. Let's plot the probability matrix together and see what will happen!

In [ ]:
%matplotlib inline
# set figure size
plt.subplots(figsize=(10, 5))
plot_probability_alive_matrix(bgf)
plt.show()

# 5. Train Gamma Gamma Model (Expected average spending)

Now, we will train the gamma-gamma model to estimate average spending for future transactional events. Then, it will be used for calculating the customer lifetime value. 

In [ ]:
# Creating Gamma-Gamma Model
ggf = GammaGammaFitter(penalizer_coef=0.01) # model object
ggf.fit(cltv['Frequency'], cltv['Monetary']) # model fitting

# Prediction of expected amount of average profit
cltv["expct_avg_spend"] = ggf.conditional_expected_average_profit(cltv['Frequency'], cltv['Monetary'])

cltv.head()

## 6. Final: Calculate CLTV

In [ ]:
# Calculating customer lifetime value by using BG-NBD and GammaGamma models: 

cltv["cltv_one_year"] = ggf.customer_lifetime_value(bgf,
                                   cltv['Frequency'],
                                   cltv['Recency'],
                                   cltv['T'],
                                   cltv['Monetary'],
                                   time=12,  # 12 month
                                   freq="D",  # frequency of T
                                   discount_rate=0.01)

cltv.sort_values("cltv_one_year",ascending=False).head()

<p>Together, we get the most valuable five customers for a year! </p>
<p>This model is capable of producing valuable information to be used for several purposes by various departments such as marketing, sales, and product development. By using it, we can simply create segments according to the CLTV. We can also extract the least valuable customers and send them to customer relationship management to increase customer engagement with the company.

Thank you for making it to the very end of this notebook!

Please feel free to leave a comment to improve it together. 